## Utility Functions

In [1]:
import random
import numpy as np

def linear(x):
    return x

def relu(x):
    return np.maximum(x, 0)

def sigmoid(x):
    return 1.0/(1.0 + np.exp(-x))

def softmax(x):
    e_x = np.exp(x - np.max(x))
    return e_x / e_x.sum(axis=0)

def get_random_pale_color():
    """
    Generates a random pale color in hexadecimal format.

    :return: A random pale color.
    """
    r = random.randint(200, 255)
    g = random.randint(200, 255)
    b = random.randint(200, 255)
    return '#%02x%02x%02x' % (r, g, b)


## FFNN Implementation

In [2]:
import numpy as np

class FFNNLayer:
    def __init__(self, number_of_neurons: int, activation_function: str):
        """
        :param number_of_neurons:
        :param activation_function:
        """
        self.number_of_neurons = number_of_neurons
        self.activation_function = activation_function


class FFNN:
    def __init__(self, input_size: int, layers: list, weights: list):
        """
        initializes the model

        :param input_size: the size of input to be received
        :param layers: list of FFNNLayer that specifies each layer's neuron count and activation function
        :param weights: list of weight on each layer
        """
        self.input_size = input_size
        self.number_of_layers = len(layers)
        self.layers = layers
        self.X = []
        self.Y = []
        self.prediction = None
        self.Y_expected = None
        self.weights = [np.array(l) for l in weights]

    def fit(self, x:list, y_expected=None):
        """
        fit the test x and y_expected to the model

        :param x: X inputs to be predicted
        :param y_expected: expected y
        :return: void
        """
        if y_expected is None:
            y_expected = []
        self.X = x
        self.Y_expected = y_expected

    def predict(self):
        """
        calculates the output by doing forward propagation

        :return: output of each X
        """
        res = self.X
        for i in range(self.number_of_layers):
            res = [np.insert(x, 0, 1) for x in res]
            net = [np.matmul(x, self.weights[i]) for x in res]
            act_func = self.layers[i].activation_function
            if act_func == 'linear':
                res = [linear(x) for x in net]
            if act_func == 'relu':
                res = [relu(n) for n in net]
            if act_func == 'sigmoid':
                res = [sigmoid(n) for n in net]
            if act_func =="softmax":
                res = [softmax(n) for n in net]
        self.prediction = res
        return res

    def calculate_sse(self):
        """
        calculate Sum Squared Error (SSE) of result and expected

        :return: sum squared error
        """
        expected = np.array(self.Y_expected[0])
        squared_error = (expected - self.prediction[0]) ** 2
        sum_squared_error = np.sum(squared_error)
        return sum_squared_error
    
    def print_expected_output(self):
        """
        prints the expected output in a formatted way.

        :return: void
        """
        print(f"Expected Output:")
        for i, sublist in enumerate(self.Y_expected):
            for j, val in enumerate(sublist):
                print(f"Output {i+1}.{j+1}: {val:.4f}")
        print("-" * 20)  # Separator 

    def print_prediction_results(self):
        """
        prints the prediction results in a formatted way.

        :return: void
        """
        print(f"Prediction Result:")
        result = [[float(format(val, '.4f')) for val in array] for array in self.prediction]
        print(result)
        for i, result in enumerate(self.prediction):
            print(f"Input {i+1}:")
            if isinstance(result, np.ndarray):
                for j, val in enumerate(result.flatten()): 
                    print(f"  Output {i+1}.{j+1}: {val:.4f}")
            else:
                print(f"  Output: {result:.4f}")
            print("-" * 20)  # Separator 
        

## FFNNVisualization Implementation

In [3]:
from collections import namedtuple
from graphviz import Digraph

class FFNNVisualizer:
    """
    Visualizer for Feed-Forward Neural Network (FFNN).
    """
    def __init__(self, ffnn: FFNN):
        """
        Initializes the visualizer with the given FFNN.

        :param ffnn: The FFNN to be visualized.
        """
        self.ffnn = ffnn

        # Create a mock input layer and insert it at the beginning of the layers list
        input_layer = self._create_mock_input_layer()
        self.layers = [input_layer] + self.ffnn.layers

    def _create_mock_input_layer(self):
        """
        Creates a mock input layer with the number of neurons equal to the input size of the FFNN.

        :return: A mock input layer.
        """
        MockLayer = namedtuple('InputLayer', ['number_of_neurons'])
        return MockLayer(self.ffnn.input_size)

    def visualize(self, output_path: str = 'bin/FFNN'):
        """
        Visualizes the FFNN by creating a graph with nodes and edges representing neurons and connections.

        :param output_path: The path where the output image will be saved.
        """
        dot = Digraph(format='png')
        dot.attr(ranksep='3')  # Set the distance between layers
        dot.attr(nodesep='0.5')  # Set the distance between node in one layer

        # Add nodes and edges for each layer
        for i, layer in enumerate(self.layers):
            with dot.subgraph(name=f'cluster_{i}') as c:
                c.attr(color='white')
                self._add_nodes_to_graph(c, i, layer)
                if i > 0:
                    self._add_edges_to_graph(dot, i)

        # Render the graph and save it to the output path
        dot.render(output_path, view=True)

    def _add_nodes_to_graph(self, graph, layer_index, layer):
        num_neurons = layer.number_of_neurons if layer_index == len(
            self.layers) - 1 else layer.number_of_neurons + 1

        # Get a random pale color for the layer
        color = get_random_pale_color()

        # Add a node for each neuron
        for j in range(num_neurons):
            node_name = self._get_node_name(layer_index, j)
            graph.node(node_name, fillcolor=color, style='filled')

    def _get_node_name(self, layer_index, neuron_index):
        """
        Generates a name for a node based on its layer and neuron indices.

        :param layer_index: The index of the layer.
        :param neuron_index: The index of the neuron within its layer.
        :return: The name of the node.
        """
        if layer_index == 0:
            return 'I' + str(neuron_index)
        elif layer_index == len(self.layers) - 1:
            return 'O' + str(neuron_index + 1)
        else:
            return 'H' + str(layer_index) + str(neuron_index)

    def _add_edges_to_graph(self, graph, layer_index):
        """
        Adds edges to the given graph for each connection between the neurons in the current and previous layers.

        :param graph: The graph to which edges will be added.
        :param layer_index: The index of the current layer.
        """
        is_last_layer = layer_index == len(self.layers) - 1

        # Add an edge for each connection
        for j in range(self.layers[layer_index - 1].number_of_neurons + 1):
            source_node_name = self._get_node_name(layer_index - 1, j)
            for k in range(self.layers[layer_index].number_of_neurons):
                weight = self.ffnn.weights[layer_index - 1][j][k]
                target_node_name = self._get_node_name(layer_index, k if is_last_layer else k + 1)
                graph.edge(source_node_name, target_node_name, label=str(weight))


## Main

In [4]:
import json
file_path = input("Enter json file path: ")
f = open(file_path)
data = json.load(f)

try:
  data_layers = data["case"]["model"]["layers"]
  layers = []
  for layer in data_layers:
    activation_func = layer["activation_function"]
    if activation_func not in ["linear", "relu", "sigmoid", "softmax"]:
      raise Exception("Activation function " + activation_func + " not available")
    layers.append(FFNNLayer(layer["number_of_neurons"], activation_func))

  weights = data["case"]["weights"]
  input_size = data["case"]["model"]["input_size"]
  input_x = data["case"]["input"]
  expected_output = data["expect"]["output"]

  model = FFNN(input_size, layers, weights)
  model.fit(input_x, expected_output)
  model.print_expected_output()
  visualizer = FFNNVisualizer(model)
  visualizer.visualize()

  result = model.predict()
  model.print_prediction_results()

  max_sse = data["expect"]["max_sse"]
  sse = model.calculate_sse()
  print(f"Sum Squared Error: {sse:.4f}")
  if sse < max_sse:
      print("Sum Squared Error(SSE) of prediction is lower than Maximum SSE")
  else:
      print("Sum Squared Error(SSE) of prediction surpass the Maximum SSE")
except KeyError as ke:
  print('Key', ke, "not found in json data. Please check your json data format")
except Exception as error:
  print("An exception occurred: ", error)


Expected Output:
Output 1.1: -11.0000
Output 2.1: -8.0000
Output 3.1: -5.0000
Output 4.1: -2.0000
Output 5.1: 1.0000
Output 6.1: 4.0000
Output 7.1: 7.0000
Output 8.1: 10.0000
Output 9.1: 13.0000
Output 10.1: 16.0000
--------------------
Prediction Result:
[[-11.0], [-8.0], [-5.0], [-2.0], [1.0], [4.0], [7.0], [10.0], [13.0], [16.0]]
Input 1:
  Output 1.1: -11.0000
--------------------
Input 2:
  Output 2.1: -8.0000
--------------------
Input 3:
  Output 3.1: -5.0000
--------------------
Input 4:
  Output 4.1: -2.0000
--------------------
Input 5:
  Output 5.1: 1.0000
--------------------
Input 6:
  Output 6.1: 4.0000
--------------------
Input 7:
  Output 7.1: 7.0000
--------------------
Input 8:
  Output 8.1: 10.0000
--------------------
Input 9:
  Output 9.1: 13.0000
--------------------
Input 10:
  Output 10.1: 16.0000
--------------------
Sum Squared Error: 0.0000
Sum Squared Error(SSE) of prediction is lower than Maximum SSE
